<b> Sales Prediction using Recurrent Nueral Net </b>

Predictions can be done using any of the ML processes. It can involve simplistic prediction using Linear regression to more complex algos which involve Multi Layer Perceptrons(MLP) or RNN's (Recurrent Nueral Networks). The key to getting predictions close to the actuals depends mostly on Feature Engineering. Generally we have limited number of features available to predict however the skill lies in constructing derived features using the given base features

This is a part of the Kaggle competetion and a good example for Prediction of Sales or Sales Forecasting using Recurrent Nueral Net. Following files are available for the purpose of forecasting <br>
1) Sales_Train_Validation.csv : This file contains historic sales data for Walmart chain for stores across 3 states and having data for 10K products across stores and states <br>
2) Calendar.csv : This file has all the days information, holiday info, special day info etc <br>
3) Selling_Price.csv : Contains Info pertaining to selling price of the products <br>

The Sales_Train_Validation is kind of POS data collected by all the store branches. This is the most important piece of data we have to create effective features for prediction.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train=pd.read_csv('C:/Kaggle/Walmart contest/sales_train_validation.csv')

<b>Sales Validation Data Frame:</b><br>
    id : unique_id to identify the product <br>
    item_id : unique id of item <br>
    dept_id : unique dept identification <br>
    cat_id : unique category_id <br>
    store_id : unique id of store <br>
    state_id : unique id per state <br>
    d1 to d1913 : Sales per day. d1 being 1st day to 1913 being that day number. Overall its 5 year data set. <br>
    
The daily sales file gives us only one feature directly which is sales per day, however that migh not be enough from prediction perspective. We will need to expand on it. A few more features that can be directly derived from this data would be 
    1) Mean SalesQty per item_id
    2) Mean SalesQty per dept_id 
    3) Mean SalesQty per cat_id 
    4) Mean SalesQty per store_id 
    5) Mean SalesQty per state_id
    
Other than the central values, Spread values in form of Standard deviation can also be derived and represented as feature. One obvious disadvantage that I have observed using both Mean and Std as input features is that the predicted value generally lies between <b>mean +/- std </b>. Per Chebeshevs Algo though 75% of the values may lie in this range we cerrtainly miss 15 to 20 % of values that lies between <b> mean +/- 2*std </b>
    
Other than Mean and standard deviations, one important value that must be considered especially in Time Series kind of forecasting is <b>correlation</b>. Corelation gives the ability to check on the seasonality component of any time series such that we can easily establish the trend.

In [3]:
df_train

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [3]:
train_val_df=df_train.copy()

Converts data type to ensure efficient memory usage for big datasets by appropriate type conversions

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [5]:
train_val_df.drop(train_val_df.iloc[:, 6:1183], inplace=True, axis=1)

Optimize Memory

In [6]:
train_val_df=reduce_mem_usage(train_val_df)

Memory usage after optimization is: 34.31 MB
Decreased by 80.1%


The input data is in horizontal form i.e. the sales per day are in form of column name d_1,d_2...d_n. From the perspective of training we would need to spread the dates in a single columns. We can carry out all other operations in series form however from being comfortable I melted the data frame using melt.

In [7]:
train_val_df=train_val_df.melt(['id','item_id','dept_id','cat_id','store_id','state_id'],var_name="d", value_name="SalesQty")

In [8]:
len(train_val_df)

22440640

Since there are 1000 products, one way to spread them from prediction perspective is by cross tabing. However since there are 1000 products we cannot have 1000 cross tabs. So we take mean,std,max, min and median which is distinct per store and hence can be used to uniquely identify the product. If we have products or store whose data is not present and predictions are required for those then cross tabing helps.

In [9]:
len(train_val_df)

22440640

2 years dataset being considered for calculation. We have shifted datasets by 28 days, 1 quarter, 6 months and 1 year to get the autocorrelation factor in play.

In [8]:
#train_val_df_stat=train_val_df.groupby('id').agg(['mean', 'std','max','min','median'])
train_val_df_stat=train_val_df.groupby('id').agg(['max'])['SalesQty']

In [11]:
train_val_df_stat

,max,median
id,,
FOODS_1_001_CA_1_validation,7,0.0
FOODS_1_001_CA_2_validation,14,0.0
FOODS_1_001_CA_3_validation,24,0.0
FOODS_1_001_CA_4_validation,5,0.0
FOODS_1_001_TX_1_validation,24,0.0
...,...,...
HOUSEHOLD_2_516_TX_2_validation,3,0.0
HOUSEHOLD_2_516_TX_3_validation,2,0.0
HOUSEHOLD_2_516_WI_1_validation,2,0.0


In [9]:
train_val_df_stat.columns = train_val_df_stat.columns.map(lambda x: x[1])
train_val_df_stat=train_val_df_stat.reset_index()

In [10]:
train_val_df_stat.rename(columns = {'a':'max'}, inplace = True)

In [11]:
train_val_df=train_val_df.drop(['item_id','store_id'],axis=1)

In [12]:
train_val_df['prev_qty']=train_val_df.groupby('id')['SalesQty'].shift(28)
train_val_df['prev_qty_qtr']=train_val_df.groupby('id')['SalesQty'].shift(round(91))
train_val_df['prev_qty_six']=train_val_df.groupby('id')['SalesQty'].shift(round(182))
train_val_df['prev_qty_yrly']=train_val_df.groupby('id')['SalesQty'].shift(round(365))

In [13]:
train_val_df_mon=train_val_df[-914700*3:]

In [14]:
train_val_df_mon=pd.merge(train_val_df_mon, train_val_df_stat, on=['id'])

In [15]:
train_val_df_mon.sort_values(by=['d'], inplace=True)

In [113]:
train_val_df_mon

,id,dept_id,cat_id,state_id,d,SalesQty,prev_qty,prev_qty_qtr,prev_qty_six,prev_qty_yrly,max,date,wday,month,year,snap_CA,snap_TX,snap_WI,evt1,evt2
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,d_1884,-0.109697,0.708008,-0.368652,-0.092102,0.230225,-0.355941,2016-03-26,1,3,2016,0,0,0,0,0
1,FOODS_1_123_TX_3_validation,FOODS_1,FOODS,TX,d_1884,-0.109697,-0.378662,-0.368652,-0.364746,-0.352539,-0.094059,2016-03-26,1,3,2016,0,0,0,0,0
2,FOODS_1_122_TX_3_validation,FOODS_1,FOODS,TX,d_1884,0.162116,-0.378662,-0.078491,-0.092102,-0.352539,-0.225000,2016-03-26,1,3,2016,0,0,0,0,0
3,FOODS_3_658_CA_1_validation,FOODS_3,FOODS,CA,d_1884,-0.109697,-0.378662,-0.368652,-0.092102,0.230225,-0.290471,2016-03-26,1,3,2016,0,0,0,0,0
4,FOODS_1_121_TX_3_validation,FOODS_1,FOODS,TX,d_1884,0.162116,-0.106934,-0.368652,-0.364746,-0.352539,0.887997,2016-03-26,1,3,2016,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914695,HOUSEHOLD_2_286_TX_3_validation,HOUSEHOLD_2,HOUSEHOLD,TX,d_1913,-0.381510,-0.378662,-0.368652,-0.092102,-0.352539,-0.486882,2016-04-24,2,4,2016,0,0,0,0,0
914696,HOUSEHOLD_2_287_TX_3_validation,HOUSEHOLD_2,HOUSEHOLD,TX,d_1913,-0.381510,-0.378662,-0.078491,-0.364746,-0.352539,-0.290471,2016-04-24,2,4,2016,0,0,0,0,0
914697,HOUSEHOLD_2_288_TX_3_validation,HOUSEHOLD_2,HOUSEHOLD,TX,d_1913,-0.381510,-0.378662,-0.368652,-0.364746,-0.352539,-0.421411,2016-04-24,2,4,2016,0,0,0,0,0
914698,HOUSEHOLD_2_290_TX_3_validation,HOUSEHOLD_2,HOUSEHOLD,TX,d_1913,-0.381510,-0.378662,-0.368652,-0.364746,-0.352539,-0.552352,2016-04-24,2,4,2016,0,0,0,0,0


In [16]:
#train_val_df=train_val_df[-365*1000:]
train_val_df_mon=train_val_df_mon.dropna()

In [17]:
sell_price_df=pd.read_csv('C:/Kaggle/Walmart contest/sell_prices.csv')

In [18]:
sell_price_df['id']=sell_price_df['item_id'] + '_' + sell_price_df['store_id'] + '_validation'

In [219]:
sell_price_df

,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation
...,...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00,FOODS_3_827_WI_3_validation
6841117,WI_3,FOODS_3_827,11618,1.00,FOODS_3_827_WI_3_validation
6841118,WI_3,FOODS_3_827,11619,1.00,FOODS_3_827_WI_3_validation
6841119,WI_3,FOODS_3_827,11620,1.00,FOODS_3_827_WI_3_validation


RNN defined for prediction

In [19]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import datetime as dt
import numpy as np
import holidays
#import holidays
import os
import logging
import sys
from scipy import stats
import random

In [20]:
class NewRNN(nn.Module):
    
    def __init__(self, input_size, n_hidden=256, n_layers=2,num_classes=1,
                               drop_prob=0.2, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, num_classes)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        #h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        #if (train_on_gpu):
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))
        #else:
        #    hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
        #              weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
    

In [21]:
calendar_df=pd.read_csv('C://Kaggle//Walmart contest//calendar.csv')

In [143]:
train_val_df_mon

,id,dept_id,cat_id,state_id,d,SalesQty,prev_qty,prev_qty_qtr,prev_qty_six,prev_qty_yrly,max
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,d_1884,1,4.0,0.0,1.0,2.0,5
600780,FOODS_1_123_TX_3_validation,FOODS_1,FOODS,TX,d_1884,1,0.0,0.0,0.0,0.0,9
600750,FOODS_1_122_TX_3_validation,FOODS_1,FOODS,TX,d_1884,2,0.0,1.0,1.0,0.0,7
86460,FOODS_3_658_CA_1_validation,FOODS_3,FOODS,CA,d_1884,1,0.0,0.0,1.0,2.0,6
600720,FOODS_1_121_TX_3_validation,FOODS_1,FOODS,TX,d_1884,2,1.0,0.0,0.0,0.0,24
...,...,...,...,...,...,...,...,...,...,...,...
590279,HOUSEHOLD_2_286_TX_3_validation,HOUSEHOLD_2,HOUSEHOLD,TX,d_1913,0,0.0,0.0,1.0,0.0,3
590309,HOUSEHOLD_2_287_TX_3_validation,HOUSEHOLD_2,HOUSEHOLD,TX,d_1913,0,0.0,1.0,0.0,0.0,6
590339,HOUSEHOLD_2_288_TX_3_validation,HOUSEHOLD_2,HOUSEHOLD,TX,d_1913,0,0.0,0.0,0.0,0.0,4
590399,HOUSEHOLD_2_290_TX_3_validation,HOUSEHOLD_2,HOUSEHOLD,TX,d_1913,0,0.0,0.0,0.0,0.0,2


In [22]:
train_val_df=reduce_mem_usage(train_val_df)

Memory usage after optimization is: 1070.05 MB
Decreased by 32.4%


In [23]:
event_type = {'Sporting': 1,'Cultural': 2,'National':3,'Religious':4,'NA':0} 
calendar_df['event_type_1']=calendar_df['event_type_1'].fillna('NA')
calendar_df['event_type_2']=calendar_df['event_type_2'].fillna('NA')
calendar_df['evt1'] = [event_type[item] for item in calendar_df['event_type_1']] 
calendar_df['evt2'] = [event_type[item] for item in calendar_df.event_type_2] 

In [24]:
calendar_df=calendar_df.drop(columns=['event_name_1','event_name_2','event_type_1','event_type_2','weekday'],axis=1)

In [27]:
train_val_df_mon.columns

Index(['id', 'dept_id', 'cat_id', 'state_id', 'd', 'SalesQty', 'prev_qty',
       'prev_qty_qtr', 'prev_qty_six', 'prev_qty_yrly', 'max'],
      dtype='object')

In [25]:
sell_price_df

,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation
...,...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00,FOODS_3_827_WI_3_validation
6841117,WI_3,FOODS_3_827,11618,1.00,FOODS_3_827_WI_3_validation
6841118,WI_3,FOODS_3_827,11619,1.00,FOODS_3_827_WI_3_validation
6841119,WI_3,FOODS_3_827,11620,1.00,FOODS_3_827_WI_3_validation


In [29]:
train_val_df_mon=pd.merge(train_val_df_mon, sell_price_df,how='inner', on=['id','wm_yr_wk']) ###to be removed

In [28]:
train_val_df_mon=pd.merge(train_val_df_mon, calendar_df, on=['d'])

In [ ]:
train_val_df_mon=pd.merge(train_val_df_mon, sell_price_df,how='inner', on=['id','wm_yr_wk'])

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
train_val_df_mon=reduce_mem_usage(train_val_df_mon)

Memory usage after optimization is: 253.83 MB
Decreased by 48.4%


In [32]:
#quant_features=['prev_qty','prev_qty_qtr','prev_qty_six','prev_qty_yrly','SalesQty','max','min','median','mean','std']#,'SellingPrice']
quant_features=['prev_qty','prev_qty_qtr','prev_qty_six','prev_qty_yrly','SalesQty','max','sell_price']#,'SellingPrice']
scaled_features = {}
mean=0
std=0
for each in quant_features:
    mean, std = train_val_df_mon[each].astype(np.float64).mean(), train_val_df_mon[each].astype(np.float64).std()
    scaled_features[each] = [mean, std]
    train_val_df_mon.loc[:, each] = (train_val_df_mon[each] - mean)/std

In [150]:
scaled_features


{'prev_qty': [1.3937564228708865, 3.679026398983083],
 'prev_qty_qtr': [1.2702088116322292, 3.444366296162568],
 'prev_qty_six': [1.3377785066141905, 3.6678126227372405],
 'prev_qty_yrly': [1.2103564010057943, 3.4311713731767184],
 'SalesQty': [1.4035738493495136, 3.678997867809074],
 'max': [10.436667759921285, 15.274067762587052],
 'sell_price': [4.481240623593395, 3.5339491494191346]}

In [154]:
train_df.columns

Index(['prev_qty', 'prev_qty_qtr', 'prev_qty_six', 'prev_qty_yrly', 'max',
       'wm_yr_wk', 'wday', 'snap_CA', 'snap_TX', 'snap_WI', 'evt1', 'evt2',
       'sell_price'],
      dtype='object')

In [224]:
train_df,target_df=train_val_df_mon.drop(['date','id','d','year','SalesQty','dept_id','cat_id','state_id','month','item_id','store_id','wm_yr_wk'],axis=1),train_val_df_mon[['SalesQty']]

In [225]:
train_val_df_mon=train_val_df_mon.fillna(0)

In [155]:
train_df=train_df.drop(columns=['wm_yr_wk'],axis=1)

In [227]:
train_df=pd.get_dummies(train_df, columns=['wday'], prefix=['wday'])


In [228]:
train=torch.from_numpy(train_df.values).float()
target=torch.from_numpy(target_df.values).float()

In [625]:
train


tensor([[-0.4448, -0.4448, -0.4409,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0662, -0.4448, -0.4409,  ...,  0.0000,  0.0000,  0.0000],
        [-0.4448, -0.4448, -0.4409,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.4448, -0.4448, -0.4409,  ...,  0.0000,  0.0000,  0.0000],
        [-0.4448, -0.4448, -0.4409,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.6914, -0.4448,  0.3442,  ...,  0.0000,  0.0000,  0.0000]])

In [229]:
target=target.squeeze()
total_step = len(train)

In [267]:
input_size = train.shape[1]
hidden_size = 80
num_layers = 2
num_classes = 1
batch_size = 1
num_epochs = 2
learning_rate = 0.01

#model = NeuralNet(input_size, hidden_size, num_classes).to(device)
#model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

model = pickle.load(open(model_path, 'rb')).to(device)


    
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
    #num_epochs=15

for epoch in range(num_epochs):
    h = model.init_hidden(batch_size)
    for i in range(len(train)):
            #images = train[i].reshape(-1,input_size).to(device)
            # h is for RNN and reshape(-1,1,inp_size) as well
        h = tuple([each.data for each in h])
        images = train[i].reshape(-1,1,input_size).to(device)
        labels = target[i].to(device)
        
            # Forward pass      
            #outputs,h = model(images,h)
            #print(labels,target[i],outputs,labels.shape)
        outputs,h = model(images,h)
        loss = criterion(outputs, labels)
        
            # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
              .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

C:\Users\vikas\anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/2], Step [10000/2743944], Loss: 1.1282
Epoch [1/2], Step [20000/2743944], Loss: 0.0004
Epoch [1/2], Step [30000/2743944], Loss: 10.8480
Epoch [1/2], Step [40000/2743944], Loss: 0.0302
Epoch [1/2], Step [50000/2743944], Loss: 0.0090
Epoch [1/2], Step [60000/2743944], Loss: 0.0030
Epoch [1/2], Step [70000/2743944], Loss: 0.0401
Epoch [1/2], Step [80000/2743944], Loss: 0.0176
Epoch [1/2], Step [90000/2743944], Loss: 0.6382
Epoch [1/2], Step [100000/2743944], Loss: 0.0001
Epoch [1/2], Step [110000/2743944], Loss: 0.0511
Epoch [1/2], Step [120000/2743944], Loss: 0.1071
Epoch [1/2], Step [130000/2743944], Loss: 1.6442
Epoch [1/2], Step [140000/2743944], Loss: 0.2418
Epoch [1/2], Step [150000/2743944], Loss: 0.1968
Epoch [1/2], Step [160000/2743944], Loss: 0.2354
Epoch [1/2], Step [170000/2743944], Loss: 0.2698
Epoch [1/2], Step [180000/2743944], Loss: 0.0020
Epoch [1/2], Step [190000/2743944], Loss: 0.0326
Epoch [1/2], Step [200000/2743944], Loss: 0.0055
Epoch [1/2], Step [210000/27

Epoch [1/2], Step [1670000/2743944], Loss: 0.0041
Epoch [1/2], Step [1680000/2743944], Loss: 0.0048
Epoch [1/2], Step [1690000/2743944], Loss: 0.0520
Epoch [1/2], Step [1700000/2743944], Loss: 1.6222
Epoch [1/2], Step [1710000/2743944], Loss: 0.0500
Epoch [1/2], Step [1720000/2743944], Loss: 0.2538
Epoch [1/2], Step [1730000/2743944], Loss: 0.0103
Epoch [1/2], Step [1740000/2743944], Loss: 0.1273
Epoch [1/2], Step [1750000/2743944], Loss: 0.0422
Epoch [1/2], Step [1760000/2743944], Loss: 0.0049
Epoch [1/2], Step [1770000/2743944], Loss: 0.0046
Epoch [1/2], Step [1780000/2743944], Loss: 0.0780
Epoch [1/2], Step [1790000/2743944], Loss: 0.2228
Epoch [1/2], Step [1800000/2743944], Loss: 0.1977
Epoch [1/2], Step [1810000/2743944], Loss: 0.4026
Epoch [1/2], Step [1820000/2743944], Loss: 0.1623
Epoch [1/2], Step [1830000/2743944], Loss: 0.1338
Epoch [1/2], Step [1840000/2743944], Loss: 0.7795
Epoch [1/2], Step [1850000/2743944], Loss: 0.0658
Epoch [1/2], Step [1860000/2743944], Loss: 0.0023


Epoch [2/2], Step [590000/2743944], Loss: 0.0975
Epoch [2/2], Step [600000/2743944], Loss: 0.1035
Epoch [2/2], Step [610000/2743944], Loss: 0.4950
Epoch [2/2], Step [620000/2743944], Loss: 0.0050
Epoch [2/2], Step [630000/2743944], Loss: 0.0024
Epoch [2/2], Step [640000/2743944], Loss: 0.0043
Epoch [2/2], Step [650000/2743944], Loss: 0.0020
Epoch [2/2], Step [660000/2743944], Loss: 0.0925
Epoch [2/2], Step [670000/2743944], Loss: 0.0104
Epoch [2/2], Step [680000/2743944], Loss: 0.0858
Epoch [2/2], Step [690000/2743944], Loss: 0.0173
Epoch [2/2], Step [700000/2743944], Loss: 0.2346
Epoch [2/2], Step [710000/2743944], Loss: 0.0116
Epoch [2/2], Step [720000/2743944], Loss: 0.2061
Epoch [2/2], Step [730000/2743944], Loss: 0.0067
Epoch [2/2], Step [740000/2743944], Loss: 0.0567
Epoch [2/2], Step [750000/2743944], Loss: 1.0385
Epoch [2/2], Step [760000/2743944], Loss: 0.0930
Epoch [2/2], Step [770000/2743944], Loss: 0.2909
Epoch [2/2], Step [780000/2743944], Loss: 0.3779
Epoch [2/2], Step [7

Epoch [2/2], Step [2240000/2743944], Loss: 0.1806
Epoch [2/2], Step [2250000/2743944], Loss: 0.0021
Epoch [2/2], Step [2260000/2743944], Loss: 2.0270
Epoch [2/2], Step [2270000/2743944], Loss: 0.0171
Epoch [2/2], Step [2280000/2743944], Loss: 0.0285
Epoch [2/2], Step [2290000/2743944], Loss: 0.1906
Epoch [2/2], Step [2300000/2743944], Loss: 0.0562
Epoch [2/2], Step [2310000/2743944], Loss: 0.7549
Epoch [2/2], Step [2320000/2743944], Loss: 0.1203
Epoch [2/2], Step [2330000/2743944], Loss: 0.2110
Epoch [2/2], Step [2340000/2743944], Loss: 1.1828
Epoch [2/2], Step [2350000/2743944], Loss: 0.0009
Epoch [2/2], Step [2360000/2743944], Loss: 0.1852
Epoch [2/2], Step [2370000/2743944], Loss: 0.0679
Epoch [2/2], Step [2380000/2743944], Loss: 2.3494
Epoch [2/2], Step [2390000/2743944], Loss: 0.0845
Epoch [2/2], Step [2400000/2743944], Loss: 0.0139
Epoch [2/2], Step [2410000/2743944], Loss: 0.0962
Epoch [2/2], Step [2420000/2743944], Loss: 0.0003
Epoch [2/2], Step [2430000/2743944], Loss: 0.1335


In [230]:
input_size = train.shape[1]
hidden_size = 80
num_layers = 2
num_classes = 1
batch_size = 1
num_epochs = 2
learning_rate = 0.01

#model = NeuralNet(input_size, hidden_size, num_classes).to(device)
#model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

model = NewRNN(input_size, hidden_size, num_layers).to(device)

    
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
    #num_epochs=15

for epoch in range(num_epochs):
    h = model.init_hidden(batch_size)
    for i in range(len(train)):
            #images = train[i].reshape(-1,input_size).to(device)
            # h is for RNN and reshape(-1,1,inp_size) as well
        h = tuple([each.data for each in h])
        images = train[i].reshape(-1,1,input_size).to(device)
        labels = target[i].to(device)
        
            # Forward pass      
            #outputs,h = model(images,h)
            #print(labels,target[i],outputs,labels.shape)
        outputs,h = model(images,h)
        loss = criterion(outputs, labels)
        
            # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
              .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

C:\Users\vikas\anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/2], Step [10000/2743944], Loss: 0.6600
Epoch [1/2], Step [20000/2743944], Loss: 0.0082
Epoch [1/2], Step [30000/2743944], Loss: 19.7622
Epoch [1/2], Step [40000/2743944], Loss: 0.1204
Epoch [1/2], Step [50000/2743944], Loss: 0.0004
Epoch [1/2], Step [60000/2743944], Loss: 0.0034
Epoch [1/2], Step [70000/2743944], Loss: 0.4243
Epoch [1/2], Step [80000/2743944], Loss: 0.0568
Epoch [1/2], Step [90000/2743944], Loss: 0.1400
Epoch [1/2], Step [100000/2743944], Loss: 0.0025
Epoch [1/2], Step [110000/2743944], Loss: 0.0064
Epoch [1/2], Step [120000/2743944], Loss: 0.0332
Epoch [1/2], Step [130000/2743944], Loss: 2.2281
Epoch [1/2], Step [140000/2743944], Loss: 0.1106
Epoch [1/2], Step [150000/2743944], Loss: 0.0346
Epoch [1/2], Step [160000/2743944], Loss: 0.5885
Epoch [1/2], Step [170000/2743944], Loss: 0.2945
Epoch [1/2], Step [180000/2743944], Loss: 0.1962
Epoch [1/2], Step [190000/2743944], Loss: 0.1542
Epoch [1/2], Step [200000/2743944], Loss: 0.1789
Epoch [1/2], Step [210000/27

Epoch [1/2], Step [1670000/2743944], Loss: 0.0155
Epoch [1/2], Step [1680000/2743944], Loss: 0.0165
Epoch [1/2], Step [1690000/2743944], Loss: 0.0036
Epoch [1/2], Step [1700000/2743944], Loss: 1.5878
Epoch [1/2], Step [1710000/2743944], Loss: 0.0462
Epoch [1/2], Step [1720000/2743944], Loss: 0.0002
Epoch [1/2], Step [1730000/2743944], Loss: 0.0002
Epoch [1/2], Step [1740000/2743944], Loss: 0.0379
Epoch [1/2], Step [1750000/2743944], Loss: 0.0438
Epoch [1/2], Step [1760000/2743944], Loss: 0.0721
Epoch [1/2], Step [1770000/2743944], Loss: 0.0022
Epoch [1/2], Step [1780000/2743944], Loss: 0.0014
Epoch [1/2], Step [1790000/2743944], Loss: 0.0593
Epoch [1/2], Step [1800000/2743944], Loss: 0.0228
Epoch [1/2], Step [1810000/2743944], Loss: 1.1323
Epoch [1/2], Step [1820000/2743944], Loss: 0.0011
Epoch [1/2], Step [1830000/2743944], Loss: 0.0026
Epoch [1/2], Step [1840000/2743944], Loss: 0.0630
Epoch [1/2], Step [1850000/2743944], Loss: 0.1204
Epoch [1/2], Step [1860000/2743944], Loss: 0.0228


Epoch [2/2], Step [590000/2743944], Loss: 0.0746
Epoch [2/2], Step [600000/2743944], Loss: 0.0072
Epoch [2/2], Step [610000/2743944], Loss: 0.0250
Epoch [2/2], Step [620000/2743944], Loss: 0.0388
Epoch [2/2], Step [630000/2743944], Loss: 0.0046
Epoch [2/2], Step [640000/2743944], Loss: 0.0002
Epoch [2/2], Step [650000/2743944], Loss: 0.0122
Epoch [2/2], Step [660000/2743944], Loss: 0.0490
Epoch [2/2], Step [670000/2743944], Loss: 0.4827
Epoch [2/2], Step [680000/2743944], Loss: 0.0011
Epoch [2/2], Step [690000/2743944], Loss: 0.0012
Epoch [2/2], Step [700000/2743944], Loss: 0.1868
Epoch [2/2], Step [710000/2743944], Loss: 0.0590
Epoch [2/2], Step [720000/2743944], Loss: 0.2069
Epoch [2/2], Step [730000/2743944], Loss: 0.0072
Epoch [2/2], Step [740000/2743944], Loss: 0.0189
Epoch [2/2], Step [750000/2743944], Loss: 0.0064
Epoch [2/2], Step [760000/2743944], Loss: 0.0021
Epoch [2/2], Step [770000/2743944], Loss: 0.0058
Epoch [2/2], Step [780000/2743944], Loss: 0.0091
Epoch [2/2], Step [7

Epoch [2/2], Step [2240000/2743944], Loss: 0.0815
Epoch [2/2], Step [2250000/2743944], Loss: 0.1315
Epoch [2/2], Step [2260000/2743944], Loss: 0.8153
Epoch [2/2], Step [2270000/2743944], Loss: 0.1554
Epoch [2/2], Step [2280000/2743944], Loss: 0.0993
Epoch [2/2], Step [2290000/2743944], Loss: 0.0431
Epoch [2/2], Step [2300000/2743944], Loss: 0.0008
Epoch [2/2], Step [2310000/2743944], Loss: 0.6206
Epoch [2/2], Step [2320000/2743944], Loss: 0.0017
Epoch [2/2], Step [2330000/2743944], Loss: 0.0080
Epoch [2/2], Step [2340000/2743944], Loss: 0.0153
Epoch [2/2], Step [2350000/2743944], Loss: 0.0000
Epoch [2/2], Step [2360000/2743944], Loss: 0.1721
Epoch [2/2], Step [2370000/2743944], Loss: 0.3337
Epoch [2/2], Step [2380000/2743944], Loss: 1.2451
Epoch [2/2], Step [2390000/2743944], Loss: 0.1421
Epoch [2/2], Step [2400000/2743944], Loss: 0.0569
Epoch [2/2], Step [2410000/2743944], Loss: 0.1029
Epoch [2/2], Step [2420000/2743944], Loss: 0.2554
Epoch [2/2], Step [2430000/2743944], Loss: 0.1410


In [268]:
import pickle
model_path='C:/Kaggle/Walmart contest//walmartpred_normalized.pkl'
pickle.dump(model, open(model_path, "wb"), pickle.HIGHEST_PROTOCOL)
hidden_layer=h

In [35]:
import pickle
model_path='C:/Kaggle/Walmart contest//walmartpred_normalized.pkl'

In [269]:
torch.save(hidden_layer,'C:/Kaggle/Walmart contest/hidden_layer.pt') #and torch.load('file.pt')

In [36]:
model=pickle.load(open(model_path, 'rb'))

In [38]:
hidden_layer=torch.load('C:/Kaggle/Walmart contest/hidden_layer.pt')

In [39]:
salesqty=list(train_val_df['SalesQty'])

In [58]:
len(salesqty)

22440640

In [46]:
newsales=list(df[:853720]['SalesQty'])

In [63]:
len(newsales)

853720

In [47]:
finalsales=salesqty+newsales

In [65]:
finalsales[23294359]

1

In [48]:
prev_qty=finalsales[:len(finalsales)-853720]
prev_qty_qtr=finalsales[:len(finalsales)-(3049*10*91)]
prev_qty_six=finalsales[:len(finalsales)-(3049*10*182)]
prev_qty_yr=finalsales[:len(finalsales)-(3049*10*365)]

In [129]:
len(prev_qty)

22440640

In [132]:
prev_qty[22440638]

3

In [40]:
df=pd.read_csv('C:/Kaggle/Walmart contest/sales_train_evaluation.csv')

In [235]:
#train_val_df_stat
df['id']=df['id'].replace('evaluation','validation')

In [87]:
salesqty[]

1707440

In [41]:
df.drop(df.iloc[:, 6:1919], inplace=True, axis=1)

In [42]:
count=1942
while (count<1970):
    col_str='d_' + str(count)
    df[col_str]=0
    count+=1


In [43]:
for i in df.index:
    df['id'].iloc[i]=df['id'].iloc[i].replace('evaluation','validation')
    print(df['id'].iloc[i])

C:\Users\vikas\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


HOBBIES_1_001_CA_1_validation
HOBBIES_1_002_CA_1_validation
HOBBIES_1_003_CA_1_validation
HOBBIES_1_004_CA_1_validation
HOBBIES_1_005_CA_1_validation
HOBBIES_1_006_CA_1_validation
HOBBIES_1_007_CA_1_validation
HOBBIES_1_008_CA_1_validation
HOBBIES_1_009_CA_1_validation
HOBBIES_1_010_CA_1_validation
HOBBIES_1_011_CA_1_validation
HOBBIES_1_012_CA_1_validation
HOBBIES_1_013_CA_1_validation
HOBBIES_1_014_CA_1_validation
HOBBIES_1_015_CA_1_validation
HOBBIES_1_016_CA_1_validation
HOBBIES_1_017_CA_1_validation
HOBBIES_1_018_CA_1_validation
HOBBIES_1_019_CA_1_validation
HOBBIES_1_020_CA_1_validation
HOBBIES_1_021_CA_1_validation
HOBBIES_1_022_CA_1_validation
HOBBIES_1_023_CA_1_validation
HOBBIES_1_024_CA_1_validation
HOBBIES_1_025_CA_1_validation
HOBBIES_1_026_CA_1_validation
HOBBIES_1_027_CA_1_validation
HOBBIES_1_028_CA_1_validation
HOBBIES_1_029_CA_1_validation
HOBBIES_1_030_CA_1_validation
HOBBIES_1_031_CA_1_validation
HOBBIES_1_032_CA_1_validation
HOBBIES_1_033_CA_1_validation
HOBBIES_1_

HOBBIES_1_338_CA_1_validation
HOBBIES_1_339_CA_1_validation
HOBBIES_1_340_CA_1_validation
HOBBIES_1_341_CA_1_validation
HOBBIES_1_342_CA_1_validation
HOBBIES_1_343_CA_1_validation
HOBBIES_1_344_CA_1_validation
HOBBIES_1_345_CA_1_validation
HOBBIES_1_346_CA_1_validation
HOBBIES_1_347_CA_1_validation
HOBBIES_1_348_CA_1_validation
HOBBIES_1_349_CA_1_validation
HOBBIES_1_350_CA_1_validation
HOBBIES_1_351_CA_1_validation
HOBBIES_1_352_CA_1_validation
HOBBIES_1_353_CA_1_validation
HOBBIES_1_354_CA_1_validation
HOBBIES_1_355_CA_1_validation
HOBBIES_1_356_CA_1_validation
HOBBIES_1_357_CA_1_validation
HOBBIES_1_358_CA_1_validation
HOBBIES_1_359_CA_1_validation
HOBBIES_1_360_CA_1_validation
HOBBIES_1_361_CA_1_validation
HOBBIES_1_362_CA_1_validation
HOBBIES_1_363_CA_1_validation
HOBBIES_1_364_CA_1_validation
HOBBIES_1_365_CA_1_validation
HOBBIES_1_366_CA_1_validation
HOBBIES_1_367_CA_1_validation
HOBBIES_1_368_CA_1_validation
HOBBIES_1_369_CA_1_validation
HOBBIES_1_370_CA_1_validation
HOBBIES_1_

HOUSEHOLD_1_070_CA_1_validation
HOUSEHOLD_1_071_CA_1_validation
HOUSEHOLD_1_072_CA_1_validation
HOUSEHOLD_1_073_CA_1_validation
HOUSEHOLD_1_074_CA_1_validation
HOUSEHOLD_1_075_CA_1_validation
HOUSEHOLD_1_076_CA_1_validation
HOUSEHOLD_1_077_CA_1_validation
HOUSEHOLD_1_078_CA_1_validation
HOUSEHOLD_1_079_CA_1_validation
HOUSEHOLD_1_080_CA_1_validation
HOUSEHOLD_1_081_CA_1_validation
HOUSEHOLD_1_082_CA_1_validation
HOUSEHOLD_1_083_CA_1_validation
HOUSEHOLD_1_085_CA_1_validation
HOUSEHOLD_1_086_CA_1_validation
HOUSEHOLD_1_087_CA_1_validation
HOUSEHOLD_1_088_CA_1_validation
HOUSEHOLD_1_089_CA_1_validation
HOUSEHOLD_1_090_CA_1_validation
HOUSEHOLD_1_091_CA_1_validation
HOUSEHOLD_1_092_CA_1_validation
HOUSEHOLD_1_093_CA_1_validation
HOUSEHOLD_1_094_CA_1_validation
HOUSEHOLD_1_095_CA_1_validation
HOUSEHOLD_1_096_CA_1_validation
HOUSEHOLD_1_097_CA_1_validation
HOUSEHOLD_1_098_CA_1_validation
HOUSEHOLD_1_099_CA_1_validation
HOUSEHOLD_1_100_CA_1_validation
HOUSEHOLD_1_101_CA_1_validation
HOUSEHOL

HOUSEHOLD_1_350_CA_1_validation
HOUSEHOLD_1_351_CA_1_validation
HOUSEHOLD_1_353_CA_1_validation
HOUSEHOLD_1_354_CA_1_validation
HOUSEHOLD_1_355_CA_1_validation
HOUSEHOLD_1_356_CA_1_validation
HOUSEHOLD_1_357_CA_1_validation
HOUSEHOLD_1_358_CA_1_validation
HOUSEHOLD_1_359_CA_1_validation
HOUSEHOLD_1_360_CA_1_validation
HOUSEHOLD_1_361_CA_1_validation
HOUSEHOLD_1_362_CA_1_validation
HOUSEHOLD_1_363_CA_1_validation
HOUSEHOLD_1_364_CA_1_validation
HOUSEHOLD_1_365_CA_1_validation
HOUSEHOLD_1_366_CA_1_validation
HOUSEHOLD_1_367_CA_1_validation
HOUSEHOLD_1_368_CA_1_validation
HOUSEHOLD_1_369_CA_1_validation
HOUSEHOLD_1_370_CA_1_validation
HOUSEHOLD_1_371_CA_1_validation
HOUSEHOLD_1_372_CA_1_validation
HOUSEHOLD_1_373_CA_1_validation
HOUSEHOLD_1_374_CA_1_validation
HOUSEHOLD_1_375_CA_1_validation
HOUSEHOLD_1_376_CA_1_validation
HOUSEHOLD_1_377_CA_1_validation
HOUSEHOLD_1_378_CA_1_validation
HOUSEHOLD_1_379_CA_1_validation
HOUSEHOLD_1_380_CA_1_validation
HOUSEHOLD_1_381_CA_1_validation
HOUSEHOL

...

HOUSEHOLD_2_466_WI_3_validation
HOUSEHOLD_2_467_WI_3_validation
HOUSEHOLD_2_468_WI_3_validation
HOUSEHOLD_2_469_WI_3_validation
HOUSEHOLD_2_470_WI_3_validation
HOUSEHOLD_2_471_WI_3_validation
HOUSEHOLD_2_472_WI_3_validation
HOUSEHOLD_2_473_WI_3_validation
HOUSEHOLD_2_474_WI_3_validation
HOUSEHOLD_2_475_WI_3_validation
HOUSEHOLD_2_476_WI_3_validation
HOUSEHOLD_2_477_WI_3_validation
HOUSEHOLD_2_478_WI_3_validation
HOUSEHOLD_2_479_WI_3_validation
HOUSEHOLD_2_480_WI_3_validation
HOUSEHOLD_2_481_WI_3_validation
HOUSEHOLD_2_482_WI_3_validation
HOUSEHOLD_2_483_WI_3_validation
HOUSEHOLD_2_484_WI_3_validation
HOUSEHOLD_2_485_WI_3_validation
HOUSEHOLD_2_486_WI_3_validation
HOUSEHOLD_2_487_WI_3_validation
HOUSEHOLD_2_488_WI_3_validation
HOUSEHOLD_2_489_WI_3_validation
HOUSEHOLD_2_490_WI_3_validation
HOUSEHOLD_2_491_WI_3_validation
HOUSEHOLD_2_492_WI_3_validation
HOUSEHOLD_2_493_WI_3_validation
HOUSEHOLD_2_494_WI_3_validation
HOUSEHOLD_2_495_WI_3_validation
HOUSEHOLD_2_496_WI_3_validation
HOUSEHOL

FOODS_2_087_WI_3_validation
FOODS_2_088_WI_3_validation
FOODS_2_089_WI_3_validation
FOODS_2_090_WI_3_validation
FOODS_2_091_WI_3_validation
FOODS_2_092_WI_3_validation
FOODS_2_093_WI_3_validation
FOODS_2_094_WI_3_validation
FOODS_2_095_WI_3_validation
FOODS_2_096_WI_3_validation
FOODS_2_097_WI_3_validation
FOODS_2_099_WI_3_validation
FOODS_2_100_WI_3_validation
FOODS_2_101_WI_3_validation
FOODS_2_102_WI_3_validation
FOODS_2_103_WI_3_validation
FOODS_2_104_WI_3_validation
FOODS_2_105_WI_3_validation
FOODS_2_106_WI_3_validation
FOODS_2_107_WI_3_validation
FOODS_2_108_WI_3_validation
FOODS_2_109_WI_3_validation
FOODS_2_110_WI_3_validation
FOODS_2_111_WI_3_validation
FOODS_2_112_WI_3_validation
FOODS_2_113_WI_3_validation
FOODS_2_114_WI_3_validation
FOODS_2_115_WI_3_validation
FOODS_2_116_WI_3_validation
FOODS_2_117_WI_3_validation
FOODS_2_118_WI_3_validation
FOODS_2_119_WI_3_validation
FOODS_2_120_WI_3_validation
FOODS_2_121_WI_3_validation
FOODS_2_122_WI_3_validation
FOODS_2_123_WI_3_val

FOODS_3_031_WI_3_validation
FOODS_3_032_WI_3_validation
FOODS_3_033_WI_3_validation
FOODS_3_034_WI_3_validation
FOODS_3_035_WI_3_validation
FOODS_3_036_WI_3_validation
FOODS_3_037_WI_3_validation
FOODS_3_038_WI_3_validation
FOODS_3_039_WI_3_validation
FOODS_3_040_WI_3_validation
FOODS_3_041_WI_3_validation
FOODS_3_042_WI_3_validation
FOODS_3_043_WI_3_validation
FOODS_3_044_WI_3_validation
FOODS_3_045_WI_3_validation
FOODS_3_046_WI_3_validation
FOODS_3_047_WI_3_validation
FOODS_3_048_WI_3_validation
FOODS_3_049_WI_3_validation
FOODS_3_050_WI_3_validation
FOODS_3_051_WI_3_validation
FOODS_3_053_WI_3_validation
FOODS_3_054_WI_3_validation
FOODS_3_055_WI_3_validation
FOODS_3_056_WI_3_validation
FOODS_3_057_WI_3_validation
FOODS_3_058_WI_3_validation
FOODS_3_059_WI_3_validation
FOODS_3_060_WI_3_validation
FOODS_3_061_WI_3_validation
FOODS_3_062_WI_3_validation
FOODS_3_063_WI_3_validation
FOODS_3_064_WI_3_validation
FOODS_3_065_WI_3_validation
FOODS_3_066_WI_3_validation
FOODS_3_067_WI_3_val

FOODS_3_418_WI_3_validation
FOODS_3_419_WI_3_validation
FOODS_3_420_WI_3_validation
FOODS_3_421_WI_3_validation
FOODS_3_422_WI_3_validation
FOODS_3_423_WI_3_validation
FOODS_3_424_WI_3_validation
FOODS_3_425_WI_3_validation
FOODS_3_426_WI_3_validation
FOODS_3_427_WI_3_validation
FOODS_3_428_WI_3_validation
FOODS_3_429_WI_3_validation
FOODS_3_430_WI_3_validation
FOODS_3_431_WI_3_validation
FOODS_3_432_WI_3_validation
FOODS_3_433_WI_3_validation
FOODS_3_434_WI_3_validation
FOODS_3_435_WI_3_validation
FOODS_3_436_WI_3_validation
FOODS_3_437_WI_3_validation
FOODS_3_438_WI_3_validation
FOODS_3_439_WI_3_validation
FOODS_3_440_WI_3_validation
FOODS_3_441_WI_3_validation
FOODS_3_442_WI_3_validation
FOODS_3_443_WI_3_validation
FOODS_3_444_WI_3_validation
FOODS_3_445_WI_3_validation
FOODS_3_446_WI_3_validation
FOODS_3_447_WI_3_validation
FOODS_3_448_WI_3_validation
FOODS_3_449_WI_3_validation
FOODS_3_450_WI_3_validation
FOODS_3_451_WI_3_validation
FOODS_3_452_WI_3_validation
FOODS_3_453_WI_3_val

FOODS_3_771_WI_3_validation
FOODS_3_772_WI_3_validation
FOODS_3_773_WI_3_validation
FOODS_3_774_WI_3_validation
FOODS_3_775_WI_3_validation
FOODS_3_776_WI_3_validation
FOODS_3_777_WI_3_validation
FOODS_3_778_WI_3_validation
FOODS_3_779_WI_3_validation
FOODS_3_780_WI_3_validation
FOODS_3_781_WI_3_validation
FOODS_3_782_WI_3_validation
FOODS_3_783_WI_3_validation
FOODS_3_784_WI_3_validation
FOODS_3_785_WI_3_validation
FOODS_3_786_WI_3_validation
FOODS_3_787_WI_3_validation
FOODS_3_788_WI_3_validation
FOODS_3_789_WI_3_validation
FOODS_3_790_WI_3_validation
FOODS_3_791_WI_3_validation
FOODS_3_792_WI_3_validation
FOODS_3_793_WI_3_validation
FOODS_3_794_WI_3_validation
FOODS_3_795_WI_3_validation
FOODS_3_796_WI_3_validation
FOODS_3_797_WI_3_validation
FOODS_3_798_WI_3_validation
FOODS_3_799_WI_3_validation
FOODS_3_800_WI_3_validation
FOODS_3_801_WI_3_validation
FOODS_3_802_WI_3_validation
FOODS_3_803_WI_3_validation
FOODS_3_804_WI_3_validation
FOODS_3_805_WI_3_validation
FOODS_3_806_WI_3_val

In [44]:
df=df.melt(['id','item_id','dept_id','cat_id','store_id','state_id'],var_name="d", value_name="SalesQty")

In [45]:
df['prev_qty']=0
df['prev_qty_qtr']=0
df['prev_qty_six']=0
df['prev_qty_yrly']=0

In [49]:
newsales=list(df[:853720]['SalesQty'])
finalsales=salesqty+newsales
prev_qty=finalsales[:len(finalsales)-853720]
prev_qty_qtr=finalsales[:len(finalsales)-(3049*10*91)]
prev_qty_six=finalsales[:len(finalsales)-(3049*10*182)]
prev_qty_yr=finalsales[:len(finalsales)-(3049*10*365)]


In [139]:
df['prev_qty']

0          0
1          0
2          0
3          0
4          1
          ..
1707435    1
1707436    0
1707437    0
1707438    3
1707439    0
Name: prev_qty, Length: 1707440, dtype: int64

In [50]:
df['prev_qty']=prev_qty[-1707440:]
df['prev_qty_qtr']=prev_qty_qtr[-1707440:]
df['prev_qty_six']=prev_qty_six[-1707440:]
df['prev_qty_yrly']=prev_qty_yr[-1707440:]

In [51]:
#pred_df=pd.merge(df, train_val_df_stat, left_index=True, right_index=True)
pred_df=pd.merge(df, train_val_df_stat, on=['id'])

In [52]:
pred_actual=pred_df.copy()

In [53]:
pred_actual=pd.merge(pred_actual, calendar_df, on=['d'])

In [54]:
pred_actual=pd.merge(pred_actual, sell_price_df, on=['id','wm_yr_wk'])

In [55]:
pred_actual=pd.get_dummies(pred_actual, columns=['wday'], prefix=['wday'])

In [56]:
#quant_features=['prev_qty','prev_qty_qtr','prev_qty_six','prev_qty_yrly','max','min','median','mean','std']#,'SellingPrice']
quant_features=['prev_qty','prev_qty_qtr','prev_qty_six','prev_qty_yrly','max','sell_price']
scaled_features_pred = {}
mean=0
std=0
for each in quant_features:
    mean, std = pred_actual[each].astype(np.float64).mean(), pred_actual[each].astype(np.float64).std()
    scaled_features_pred[each] = [mean, std]
    pred_actual.loc[:, each] = (pred_actual[each] - mean)/std

In [57]:
scaled_features_pred

{'prev_qty': [1.3820063955395212, 3.6210405152258134],
 'prev_qty_qtr': [1.3198437426790985, 3.5655286620067366],
 'prev_qty_six': [1.2848779459307502, 3.5152939844432507],
 'prev_qty_yrly': [1.2112402192756406, 3.4165453870977878],
 'max': [10.436667759921285, 15.274063886200565],
 'sell_price': [4.481793210923377, 3.533537781650872]}

In [58]:
final_tgt_df,actval_df=pred_actual.drop(['date','id','d','year','month','SalesQty','dept_id','cat_id','state_id'],axis=1),pred_actual[['SalesQty']]

In [59]:
final_tgt_df=final_tgt_df.drop(columns=['item_id_x','item_id_y','store_id_x','store_id_y'],axis=1)

In [60]:
#final_tgt_df=final_tgt_df.fillna(0)
final_tgt_df=final_tgt_df.drop(columns=['wm_yr_wk'],axis=1)

In [61]:
final=torch.from_numpy(final_tgt_df.values).float()
#target=torch.from_numpy(target_df.values).float()

In [756]:
round(scaled_features['SalesQty'][0],2)

1.18

In [62]:
scaled_features_pred

{'prev_qty': [1.3820063955395212, 3.6210405152258134],
 'prev_qty_qtr': [1.3198437426790985, 3.5655286620067366],
 'prev_qty_six': [1.2848779459307502, 3.5152939844432507],
 'prev_qty_yrly': [1.2112402192756406, 3.4165453870977878],
 'max': [10.436667759921285, 15.274063886200565],
 'sell_price': [4.481793210923377, 3.533537781650872]}

In [63]:
#mean_cal, std_cal = scaled_features_pred['prev_qty']
mean_cal, std_cal = scaled_features_pred['prev_qty']
print(mean_cal)
print(std_cal)

1.3820063955395212
3.6210405152258134


In [65]:
h=hidden_layer

In [66]:
h

(tensor([[[ 0.0000e+00, -6.8276e-15,  6.6290e-24,  6.1666e-24,  4.2611e-16,
           -0.0000e+00,  4.8868e-32, -1.0000e+00, -0.0000e+00, -1.0000e+00,
           -0.0000e+00, -1.0000e+00,  1.0000e+00, -0.0000e+00,  0.0000e+00,
            0.0000e+00,  1.0000e+00,  1.0000e+00, -8.7214e-25,  1.0000e+00,
           -7.6159e-01, -1.0000e+00,  2.8648e-20,  1.0000e+00,  1.0000e+00,
           -0.0000e+00, -7.6159e-01, -7.6159e-01, -0.0000e+00, -1.0000e+00,
           -0.0000e+00, -1.0000e+00,  7.6159e-01, -1.0000e+00, -7.6159e-01,
           -1.0000e+00,  0.0000e+00,  1.0000e+00, -1.7225e-11, -0.0000e+00,
            0.0000e+00,  5.2351e-15,  2.6833e-37,  1.0000e+00,  1.0000e+00,
            6.2119e-14, -7.6159e-01, -1.0000e+00, -1.0000e+00,  1.4375e-25,
           -7.6159e-01,  0.0000e+00,  1.1262e-30,  6.3159e-39, -0.0000e+00,
            6.5761e-16,  0.0000e+00,  0.0000e+00, -8.7984e-25, -7.6159e-01,
            1.0000e+00,  1.0000e+00,  0.0000e+00,  5.5942e-20,  0.0000e+00,
           -

In [76]:
pred_actual['pred_sales']=0
#mean_cal, std_cal = scaled_features['prev_qty']
for i in range(len(final)):
            #images = train[i].reshape(-1,input_size).to(device)
            # h is for RNN and reshape(-1,1,inp_size) as well
        h = tuple([each.data for each in h])
        images = final[i].reshape(-1,1,final.shape[1]).to(device)
        #labels = target[i].to(device)
        
            # Forward pass      
            #outputs,h = model(images,h)
            #print(labels,target[i],outputs,labels.shape)
        #print(images,h)
        outputs,h = model(images,h)
        #pred_sales.append(outputs.item())
        val=outputs.item()*std_cal+mean_cal + 0.15*(outputs.item()*std_cal+mean_cal)
        pred_actual['pred_sales'].iloc[i]=round(val,0)
        if pred_actual['pred_sales'].iloc[i] < 0:
            pred_actual['pred_sales'].iloc[i]=0
            
        #print(i,round(outputs.item(),0))

C:\Users\vikas\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [177]:
pred_actual[:10]

,id,item_id,dept_id,cat_id,store_id,state_id,d,SalesQty,prev_qty,prev_qty_qtr,...,evt2,wday_1,wday_2,wday_3,wday_4,wday_5,wday_6,wday_7,pred_sales,pred_sales_1
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,-0.381660,-0.370168,...,0,0,0,1,0,0,0,0,-0.0,-0.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,-0.381660,-0.370168,...,0,0,0,1,0,0,0,0,-0.0,-0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,-0.381660,-0.089704,...,0,0,0,1,0,0,0,0,-0.0,-0.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,-0.381660,-0.370168,...,0,0,0,1,0,0,0,0,0.0,0.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,1,-0.105496,-0.370168,...,0,0,0,1,0,0,0,0,-0.0,-0.0
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,-0.381660,-0.370168,...,0,0,0,1,0,0,0,0,-0.0,-0.0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,-0.105496,-0.370168,...,0,0,0,1,0,0,0,0,-1.0,0.0
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,19,-0.381660,1.593075,...,0,0,0,1,0,0,0,0,3.0,3.0
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,-0.381660,-0.370168,...,0,0,0,1,0,0,0,0,-0.0,-0.0
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,-0.105496,-0.370168,...,0,0,0,1,0,0,0,0,0.0,0.0


In [68]:
pred_actual.to_csv('C:/Kaggle/Walmart contest/walmartresult_final_sellprice_v6.csv')

In [176]:
pred_actual['pred_sales_1']=pred_actual['pred_sales'].clip(lower=0)

In [77]:
pred_eval=pred_actual[:30490*28]

In [78]:
pred_final=pred_actual[30490*28:]

In [191]:
pred_final

,id,item_id,dept_id,cat_id,store_id,state_id,d,SalesQty,prev_qty,prev_qty_qtr,...,evt2,wday_1,wday_2,wday_3,wday_4,wday_5,wday_6,wday_7,pred_sales,pred_sales_1
853720,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,-0.105496,-0.370168,...,0,0,0,1,0,0,0,0,-0.0,-0.0
853721,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,-0.105496,-0.370168,...,0,0,0,1,0,0,0,0,-0.0,-0.0
853722,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,-0.381660,-0.370168,...,0,0,0,1,0,0,0,0,-0.0,-0.0
853723,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,-0.381660,0.190759,...,0,0,0,1,0,0,0,0,-0.0,-0.0
853724,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1942,0,-0.105496,-0.370168,...,0,0,0,1,0,0,0,0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707435,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,-0.105496,-0.370168,...,2,0,1,0,0,0,0,0,-0.0,-0.0
1707436,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,-0.381660,-0.370168,...,2,0,1,0,0,0,0,0,0.0,0.0
1707437,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,-0.381660,-0.370168,...,2,0,1,0,0,0,0,0,-1.0,0.0
1707438,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0,0.446831,-0.370168,...,2,0,1,0,0,0,0,0,-0.0,-0.0


In [71]:
pred_eval.to_csv('C:/Kaggle/Walmart contest/walmart_first_28_sellprice_v6.csv')

In [72]:
pred_final.to_csv('C:/Kaggle/Walmart contest/walmart_final_28_sellprice_v6.csv')

In [193]:
torch.save(hidden_layer,'C:/Kaggle/Walmart contest/hidden_layer.pt') #and torch.load('file.pt')

In [74]:
from sklearn.metrics import mean_squared_error

In [197]:
pred_eval['round']=round(pred_eval['pred_sales'],0)

C:\Users\vikas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
mean_squared_error(pred_eval['SalesQty'], pred_eval['pred_sales'],squared=False)

2.700950833945987

In [266]:
print(max(pred_eval['SalesQty']),max(pred_eval['pred_sales']))

196 61.0
